In [ ]:
# make csv ready to datasets fromat
import pandas as pd

d_a = pd.read_csv("data/twitter_data/labels_tweets_dropna.csv")

from sklearn.model_selection import train_test_split
d_a['label'] = d_a['Cyberbullying'] + d_a['Insult'] + d_a['Threat']
d_a['label'] = d_a['label'].apply(lambda x: 1 if x > 0 else 0)

d_a_balanced = d_a[d_a.label == 0].iloc[:5017,:].append(d_a[d_a.label ==1]).sample(frac=1)
d_a_r = d_a_balanced[['Tweet','label']]

d_train, d_test = train_test_split(d_a_r, test_size=0.2, random_state=42)
d_test1, d_val = train_test_split(d_test, test_size=0.2, random_state=42)

d_train.to_csv("data/twitter_data/tweet_train.csv",index=False)
d_test1.to_csv("data/twitter_data/tweet_test.csv",index=False)
d_val.to_csv("data/twitter_data/tweet_val.csv",index=False)

In [ ]:
from datasets import load_dataset

data_files = {"train": "data/twitter_data/tweet_train.csv", "test": "data/twitter_data/tweet_test.csv", "val": "data/twitter_data/tweet_val.csv"}
dataset = load_dataset("csv", data_files=data_files)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"]
small_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()